In [2]:
import pandas as pd


In [3]:
df1 = pd.read_csv("../Data/eth_active_addresses_hr.csv")
df2 = pd.read_csv("../Data/eth_ohlcv_fees_tx_size_fail_rate_hr.csv")
df3 = pd.read_csv("../Data/eth_staking_inflow_hr.csv")
df4 = pd.read_csv("../Data/eth_exchange_deposite_withdraw_tx_hr.csv")
df5 = pd.read_csv("../Data/eth_exchange_netflow_usd_hr.csv")
df6 = pd.read_csv("../Data/eth_RV_hr.csv")

In [4]:
for df in [df1, df2, df3, df4, df5, df6]:
    df['datetime'] = pd.to_datetime(df['datetime']).dt.tz_localize(None)

# Merge all on datetime
merged = (
    df2
    .merge(df1, on="datetime", how="left")
    .merge(df3, on="datetime", how="left")
    .merge(df4, on="datetime", how="left")
    .merge(df5, on="datetime", how="left")
    .merge(df6, on="datetime", how="left")
    .sort_values("datetime")
)

merged = merged[merged["datetime"] != "2025-10-13 00:00:00"]
merged['staking_inflow'] = merged['staking_inflow'].fillna(0)
merged.drop(columns=['realised_variance'], inplace=True)

print(merged.shape)
print(merged.head())

(8208, 18)
             datetime  onchain_volume_usd  transaction_count  avg_gas_fee_usd  \
0 2024-11-05 00:00:00        9.682857e+07              44719         1.817381   
1 2024-11-05 01:00:00        9.693977e+07              44513         1.823433   
2 2024-11-05 02:00:00        1.304534e+08              45224         1.783517   
3 2024-11-05 03:00:00        9.107739e+07              42689         1.623861   
4 2024-11-05 04:00:00        9.376725e+07              43063         1.346194   

   avg_priority_fee_usd  fail_rate_percent     open      low     high  \
0              0.738825                2.4  2398.91  2398.91  2409.56   
1              0.819646                2.3  2402.45  2384.78  2410.94   
2              0.724244                2.0  2411.90  2402.15  2413.11   
3              0.716063                1.4  2405.17  2403.79  2417.69   
4              0.550501                1.4  2418.50  2418.50  2431.30   

     close  hourly_return  active_sending_addresses  \
0  2405.

In [5]:
# Add moving averages columns
merged['RV_MA_1hr'] = merged['realised_volatility'].shift(1)
merged['RV_MA_3hr'] = merged['RV_MA_1hr'].rolling(window=3).mean()
merged['RV_MA_12hr'] = merged['RV_MA_1hr'].rolling(window=12).mean()

In [8]:
max(merged['datetime'])

Timestamp('2025-10-12 23:00:00')

In [7]:
merged.to_csv("../Data/dune_eth_hour.csv", index=False)